In [1]:
from pam import read
import pam.vehicle
import pandas as pd
from pprint import pprint
import os

# load up population
data_path = os.path.join("data", "example_data")
trips = pd.read_csv(os.path.join(data_path, 'example_travel_diaries.csv'))
attributes = pd.read_csv(os.path.join(data_path, 'example_attributes.csv'))
attributes.set_index('pid', inplace=True)
pop = read.load_travel_diary(trips, attributes)

Using tour based purpose parser (recommended)
Adding pid->hh mapping to persons_attributes from trips.
Adding home locations to persons attributes using trips attributes.
Using freq of 'None' for all persons.


In [2]:
pop = pam.read.read_matsim(
    os.path.join(data_path, 'example_plans.xml'),
    version = 12)

In [3]:
pop.stats

{'num_households': 2926,
 'num_people': 2926,
 'num_activities': 8778,
 'num_legs': 5852}

## Simple example of vehicle assignment

Below we assign vehicles to agents with a car mode in their plans. We give default vehicles or default electric vehicles to random agents. Defaults follow the same defaults as defined in MATSim's dtd files:
[vehicleDefinitions_v2.0.xsd](https://www.matsim.org/files/dtd/vehicleDefinitions_v2.0.xsd) and
[electric_vehicles_v1.dtd](https://www.matsim.org/files/dtd/electric_vehicles_v1.dtd). Saving this population to matsim files will produce these files.

In [4]:
# let's define some vehicles

default_ev_vehicle_type = pam.vehicle.VehicleType('default_EV')
big_ev_vehicle_type = pam.vehicle.VehicleType('big_EV', length=15, width=2)
default_vehicle_type = pam.vehicle.VehicleType('default_car')

In [5]:
import random

for hid, pid, p in pop.people():
    if 'car' in p.mode_classes:
        if random.random() < 0.5:
            # let's give a vehicle to this person
            if random.random() < 0.5:
                p.assign_vehicle(pam.vehicle.ElectricVehicle(pid, vehicle_type=default_ev_vehicle_type))
            else:
                p.assign_vehicle(pam.vehicle.Vehicle(pid, default_vehicle_type))

You can also change default parameters for ElectricVehicles too. You can set their `battery_capacity` and initial state of charge `initial_soc` and the charger types allowed `charger_types`.

In [6]:
pprint(pam.vehicle.ElectricVehicle('id').__dict__)

{'battery_capacity': 60,
 'charger_types': 'default',
 'id': 'id',
 'initial_soc': 60,
 'vehicle_type': VehicleType(id='defaultElectricVehicleType', length=7.5, width=1.0, networkMode='car', capacity=CapacityType(seats=4, standingRoomInPersons=0), description='personal_vehicle', passengerCarEquivalents=1.0, flowEfficiencyFactor=1.0)}


In [7]:
len(pop.electric_vehicles())

142

In [8]:
pop.electric_vehicle_charger_types()

{'default'}

In [9]:
import logging
from pam import write

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

write.write_matsim(
    pop,
    plans_path='./plans.xml',
    vehicles_dir='./',
    version=12
)

INFO:root:Population includes vehicles
INFO:root:Saving vehicles to ./
INFO:root:Writing all vehicles to ./all_vehicles.xml
INFO:root:Population includes electric vehicles
INFO:root:Writing electric vehicles to ./electric_vehicles.xml
INFO:root:Found 142 electric vehicles with unique charger types: {'default'}. Ensure you generate a chargers xml file: https://www.matsim.org/files/dtd/chargers_v1.dtd if you're running a simulation using org.matsim.contrib.ev
